# Import Python Libraries
## Libraries Used
##### Pandas
Data manipulation and analysis
##### MatPlotLib Pyplot
2D plotting
##### Numpy
Supports large, multi-dimensional arrays and matrix manipulation and high level mathematical functions on these arrays
##### Scipy Stats
Hypothesis testing

In [ ]:
# Perform library imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

### Import Data Dictionaries to Convert Codes to Descriptions

##### Sources

Non-Profit Data - NCSS Data Archive:  https://nccs-data.urban.org/dd2.php?close=1&form=BMF+08/2016

Region Data - ...

##### Non-Profit Method
* Created text files in json format within Visual Studio for each Data Dictionary
* Saved files to Resources folder
* Used pandas library to read the files into dataframes

In [ ]:
# Import Level1 Data Dictionary and display : NOTE - This is only a break out of Public Charities
file_Level1 = "./Resources/NCSSDataDictLevel1.txt"
dict_Level1 = pd.read_json(file_Level1)
dict_Level1

### Import IRS Business Master Files

##### Source
NCSS Data Archive
https://nccs-data.urban.org/data.php?ds=bmf

##### File
bmf.bm1812.csv

In [ ]:
# specify file name
gov_data_file = "../../bookish-lamp/2018_BMF.csv"

# import file
gov_data = pd.read_csv(gov_data_file, low_memory=False)

# Display resulting dataframe header
gov_data.head()

# Data Cleaning Phase
#### Review Raw Data and Eliminate Data not Required for Analysis and Junk Data
* IMPORTANT:  Read data dictionary and determine if some rows should be eliminated in step 1
  - TAXPER ending tax period for financial data - do we know these are all the same?  Do we care?
  - Out of Scope Flag - do we only want in scope data?
  - Reason why out of scope - do we care?
  - Filer: 99 filing w/in 2 years yes or no
  - ZFiler:  990 filing 0 dollars inc/assets w/in 2 years?
  - Etc.. need to review all codes and determine if some data should be removed by code (rows) first
* Remove unwanted columns - Do this step next to avoid deleting a row where one of the un-needed columns has invalid data and the columns needed have valid data
* Determine how many values in each column
* Remove NaN or missing values
* Removed unwanted rows (States that do not map to a region, Level1 O or U types)
* Other?

Questions
* Should we be using cTotRev and cAssets fields instead of ASSETS and INCOME?  The c fields are NCCS Financial Information, the two used so far are from Basic Info.  Are these the same?

In [ ]:
# Select only the columns of data we need for analysis
gov_data = gov_data[["EIN","ASSETS","INCOME","NAME","STATE","LEVEL1"]]

In [ ]:
# check the number of data rows per column
gov_data.count()

In [ ]:
# display the data read in
gov_data

In [ ]:
# drop invalid rows and display
gov_data.dropna(axis=0, how='any', inplace=True)
gov_data

In [ ]:
# Determine if rows are even yet
gov_data.count()

In [ ]:
# identify the indices of rows we want to eliminate and display how many rows are found
indexNames = gov_data[gov_data["LEVEL1"]=="O"].index
len(indexNames)

In [ ]:
# drop the rows identified and show the resulting dataframe
gov_data.drop(indexNames, inplace=True)
gov_data.head()

In [ ]:
# identify the indices of rows we want to eliminate and display how many rows
indexNames = gov_data[gov_data["LEVEL1"]=="U"].index
len(indexNames)

In [ ]:
# drop the rows identified and show the resulting dataframe
gov_data.drop(indexNames, inplace=True)
gov_data.head()

In [ ]:
# Determine the remaining count of rows and ensure our data set is full (no uneven row counts)
gov_data.count()

# Review Basic Statistics of the Remaining Data
* Run basic statistics on the numeric columns


In [ ]:
gov_data.describe()

# Create a copy of our dataset at the State Level

In [ ]:
# Create a copy of our data at the State Level
state_level1 = gov_data.copy()
state_level1

# Apply Region Data to the dataset
 
*


# Map Level 1 Labels (demo) and Save gov_data file (cleaned)

In [ ]:
x_labels1 = gov_data["LEVEL1"].map(dict_Level1["Description1"])
x_labels1

gov_data.to_csv(index=False, path_or_buf="./Output/gov_data.csv")

# Calculate Statistics by Region: Public Charity vs Private Foundation
* Calculate count, median, mean, min, max, sem

In [ ]:
# Create a dataframe that contains statistics about Income and Assets by State and Public Charity vs Private Foundation
state_level1 = state_level1.groupby(["STATE", "LEVEL1"], as_index=False).aggregate(
    {
        "INCOME":["mean", "sem", "count", "min", "max", "median"],
        "ASSETS":["mean", "sem", "count", "min", "max", "median"]
    }
)
# Display the resulting dataframe
state_level1

# Remove any invalid rows (missing data or NaN)

In [ ]:
# Remove invalid rows and display
state_level1.dropna(axis=0, how='any', inplace=True)
state_level1

# Create Two Dataframes by Region:  Public Charities, Private Foundations

In [ ]:
# Create a dataframe for Public Charities
state_PC = state_level1[state_level1["LEVEL1"] == "PC"]
state_PC = state_PC.set_index("STATE")
state_PC.head()

In [ ]:
# Create a dataframe for Private Foundations
state_PF = state_level1[state_level1["LEVEL1"] == "PF"]
state_PF = state_PF.set_index("STATE")
state_PF.head()

In [ ]:
# Plot Median Income vs Median Assets per State
plt.figure(figsize=(8,8))
plt.scatter(state_PC["ASSETS"]["median"], state_PC["INCOME"]["median"], c="b", marker='o', alpha=0.5, label="Public Charity")
plt.scatter(state_PF["ASSETS"]["median"], state_PF["INCOME"]["median"], c="g", marker='x', alpha=0.5, label="Private Foundation")
plt.title(f'Median Assets vs Median Income per State', size=16, weight='bold')
plt.xlabel('Median Assets ($)')
plt.ylabel('Median Income ($)')
plt.grid(alpha=0.5)
plt.legend(loc="best")

# Save the image
plt.savefig("./Images/MedianAssetsvsMedianIncomebyState.png")
plt.show()

# Check Null Hypothesis
1) All states have the same number of Non-Profit Organizations (Public Charities & Private Foundations Total)

2) Median Income is the same for Public Charities and Private Foundations by State

3) Median Assets is the same for Public Charities and Private Foundations by State

4) Non-Profit count is the same for Public Charities and Private Foundations by State

# Create a function for repetitive testing
This function will do the following with two populations of data:
* Create a scatter plot
* Create a histogram
* Run a Student t-test with unequal variance
* Run an ANOVA

In [ ]:
# Function to plot two populations
def displayData(index1, index2, population1, population2, label1, label2, title, xlabel, ylabel):
    
    # set the figure size
    plt.figure(figsize=(20,8))

    # Scatter Plot of Data
    plt.subplot(2,1,1)
    plt.scatter(index1, population1, marker='o', color='b', label=label1)
    plt.scatter(index2, population2, marker='x', color='g', label=label2)
    plt.title(title,color='k', size=14, weight='bold')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc="best")
    plt.grid(alpha=0.5)
    
    # Historgram Plot of Data
    plt.subplot(2, 1, 2)
    plt.hist(population1, 10, density=True, alpha=0.7, color='b', label=label1)
    plt.hist(population2, 10, density=True, alpha=0.7, color='g', label=label2)
    plt.axvline(population1.mean(), color='b', linestyle='dashed', linewidth=2)
    plt.axvline(population2.mean(), color='g', linestyle='dashed', linewidth=2)
    plt.xlabel(ylabel)
    plt.ylabel("Frequency")
    plt.legend(loc="best")
    plt.grid(alpha=0.5)
    
    # Save the Figure
    plt.savefig("./Images/" + title.replace(" ", "") + ".png")
    
    # Student t-test
    print('\033[1m' + "Student t-test with unequal variance" + '\033[0m')
    print(stats.ttest_ind(population1, population2, equal_var=False))
    
    # ANOVA test
    print('\033[1m' + '\nANOVA test' + '\033[0m')
    print(stats.f_oneway(population1, population2))
    
    return

# Null Hypothesis 1:  All states have the same number of Non-Profit Organizations
#### Where
Non-Profit in scope = Public Charities & Private Foundations

In [ ]:
# Test with a Chi-square
stats.chisquare(gov_data["STATE"].value_counts())

#### False:  All states do not have the same number of Non-Profit Organizations.

# Null Hypothesis 2:  Median Income is the same for Public Charities and Private Foundations by State

In [ ]:
# Using the dataframes by State and type of Non-Profit, run tests
displayData(state_PC.index, state_PF.index, state_PC["INCOME"]["median"], state_PF["INCOME"]["median"],
            "Public Charity", "Private Foundation", "Median Income by State", "State", "Median Income ($)")

# Null Hypothesis 3:  Median Assets are the same for Public Charities and Private Foundations by State

In [ ]:
# Using the dataframes by State and type of Non-Profit, run tests
displayData(state_PC.index, state_PF.index, state_PC["ASSETS"]["median"], state_PF["ASSETS"]["median"],
            "Public Charity", "Private Foundation", "Median Assets by State", "State", "Median Assets ($)")

# Null Hypothesis 4:  Non-Profit count is the same for Public Charities and Private Foundations by State

In [ ]:
# Using the dataframes by State and type of Non-Profit, run tests
displayData(state_PC.index, state_PF.index, state_PC["ASSETS"]["count"], state_PF["ASSETS"]["count"],
            "Public Charity", "Private Foundation", "Count of Non-Profits by State", "State", "Count of Non-Profits")